In [1]:
from __future__ import division, print_function, absolute_import

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=False)

import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [8]:
# Parameters
learning_rate = 0.001
num_steps = 2000
batch_size = 128
display_step = 20

In [13]:
# Network Parameters
n_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.5 # Dropout, probability to keep units

In [24]:
# Create the neural network
def alex_net(x_dict, num_classes, dropout, reuse, is_training):
    
    # Define a scope for reusing the variables
    with tf.variable_scope('ConvNet', reuse=reuse):
        # TF Estimator input is a dict, in case of multiple inputs
        x = x_dict['images']

        # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
        # Reshape to match picture format [Height x Width x Channel]
        # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
        x = tf.reshape(x, shape=[-1, 28, 28, 1])

        # Convolution Layer with 32 filters and a kernel size of 3
        conv1 = tf.layers.conv2d(x, 32, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv1 = tf.layers.max_pooling2d(conv1, 2, 2)
        # Batch norm
        conv1 = tf.contrib.layers.batch_norm(conv1)

        # Convolution Layer with 64 filters and a kernel size of 3
        conv2 = tf.layers.conv2d(conv1, 64, 3, activation=tf.nn.relu)
        # Max Pooling (down-sampling) with strides of 2 and kernel size of 2
        conv2 = tf.layers.max_pooling2d(conv2, 2, 2)
        # Batch Norm
        conv2 = tf.contrib.layers.batch_norm(conv2)
        
        # Convolution Layer with 64 filters and a kernel size of 3
        conv3 = tf.layers.conv2d(conv2, 64, 3, activation=tf.nn.relu)
    
        # Flatten the data to a 1-D vector for the fully connected layer
        fc1 = tf.contrib.layers.flatten(conv3)

        # Fully connected layer
        fc1 = tf.layers.dense(fc1, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        fc1 = tf.layers.dropout(fc1, rate=dropout, training=is_training)
        
        # Fully connected layer
        fc2 = tf.layers.dense(fc1, 1024)
        # Apply Dropout (if is_training is False, dropout is not applied)
        fc2 = tf.layers.dropout(fc2, rate=dropout, training=is_training)

        # Output layer, class prediction
        out = tf.layers.dense(fc2, n_classes)

    return out

In [25]:
# Define the model function (following TF Estimator Template)
def model_fn(features, labels, mode):
    
    # Build the neural network
    # Because Dropout have different behavior at training and prediction time, we
    # need to create 2 distinct computation graphs that still share the same weights.
    logits_train = alex_net(features, num_classes, dropout, reuse=False, is_training=True)
    logits_test = alex_net(features, num_classes, dropout, reuse=True, is_training=False)
    
    # Predictions
    pred_classes = tf.argmax(logits_test, axis=1)
    pred_probas = tf.nn.softmax(logits_test)
    
    # If prediction mode, early return
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=pred_classes) 
        
    # Define loss and optimizer
    loss_op = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
        logits=logits_train, labels=tf.cast(labels, dtype=tf.int32)))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss_op, global_step=tf.train.get_global_step())
    
    # Evaluate the accuracy of the model
    acc_op = tf.metrics.accuracy(labels=labels, predictions=pred_classes)
    
    # TF Estimators requires to return a EstimatorSpec, that specify
    # the different ops for training, evaluating, ...
    estim_specs = tf.estimator.EstimatorSpec(
      mode=mode,
      predictions=pred_classes,
      loss=loss_op,
      train_op=train_op,
      eval_metric_ops={'accuracy': acc_op})

    return estim_specs

In [26]:
# Build the Estimator
model = tf.estimator.Estimator(model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_log_step_count_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600, '_is_chief': True, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11a12cb38>, '_master': '', '_num_worker_replicas': 1, '_tf_random_seed': None, '_save_summary_steps': 100, '_service': None, '_task_type': 'worker', '_num_ps_replicas': 0, '_model_dir': '/var/folders/vq/_cw87bhs0ln473zqsbt7lqn40000gn/T/tmprq74xqdr', '_session_config': None, '_save_checkpoints_steps': None}


In [27]:
# Define the input function for training
input_fn = tf.estimator.inputs.numpy_input_fn(
    x={'images': mnist.train.images}, y=mnist.train.labels,
    batch_size=batch_size, num_epochs=None, shuffle=True)
# Train the Model
model.train(input_fn, steps=num_steps)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/vq/_cw87bhs0ln473zqsbt7lqn40000gn/T/tmprq74xqdr/model.ckpt.
INFO:tensorflow:step = 1, loss = 3.57852
INFO:tensorflow:global_step/sec: 4.14355
INFO:tensorflow:step = 101, loss = 0.217649 (24.135 sec)
INFO:tensorflow:global_step/sec: 3.85624
INFO:tensorflow:step = 201, loss = 0.148017 (25.932 sec)
INFO:tensorflow:global_step/sec: 4.12476
INFO:tensorflow:step = 301, loss = 0.156455 (24.244 sec)
INFO:tensorflow:global_step/sec: 3.50495
INFO:tensorflow:step = 401, loss = 0.12864 (28.532 sec)
INFO:tensorflow:global_step/sec: 3.31603
INFO:tensorflow:step = 501, loss = 0.028773 (30.159 sec)
INFO:tensorflow:global_step/sec: 3.84186
INFO:tensorflow:step = 601, loss = 0.0518613 (26.026 sec)
INFO:tensorflow:global_step/sec: 3.95874
INFO:tensorflow:step = 701, loss = 0.107985 (25.261 sec)
INFO:tensorflow:global_step/sec: 3.99589
INFO:tensorflow:step = 801, loss = 0.0503822 (25.026 sec)
INFO:tensor